In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Step 1: Fetch broker and broker branch values using Selenium
driver = webdriver.Chrome()  # Ensure you have the correct driver installed
url = "https://concords.moneydj.com/z/zc/zco/zco.djhtm?a=1101"
driver.get(url)

try:
    # Wait for the main form to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "sel_Broker"))
    )

    # Select each broker in the dropdown
    broker_select = Select(driver.find_element(By.NAME, "sel_Broker"))
    brokerDict = {}
    for broker_option in broker_select.options:
        brokerDict[broker_option.text] = broker_option.get_attribute("value")
    
    broker_branches = []
    for broker_name, broker_value in brokerDict.items():

        # Skip if value is empty
        if not broker_value:
            continue

        broker_select = Select(driver.find_element(By.NAME, "sel_Broker"))
        broker_select.select_by_value(broker_value)
        # Select each branch for the current broker
        branch_select = Select(driver.find_element(By.NAME, "sel_BrokerBranch"))

        nameValue = {}
        for branch_option in branch_select.options:
            branch_value = branch_option.get_attribute("value")
            branch_name = branch_option.text.strip()
            broker_branches.append({
                "broker_value": broker_value,
                "broker_name": broker_name,
                "branch_value": branch_value,
                "branch_name": branch_name
            })

finally:
    driver.quit()

# Step 2: Fetch table data using requests
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

def get_stock_codes():
    print('Downloading data for strMode=2')
    # Request data for strMode=2
    res_2 = requests.get("https://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    df_2 = pd.read_html(res_2.text)[0]
    df_2.columns = [
        'full_name', 'isin_code', 'listed_date', 'market_type', 'industry_type', 'cfic_code', 'remarks'
    ]
    stock_start_2 = df_2[df_2['full_name'].str.contains('股票', na=False)].index[0] + 1
    stock_end_2 = df_2[df_2['full_name'].str.contains('上市認購\(售\)權證', na=False)].index[0]
    stock_df_2 = df_2.iloc[stock_start_2:stock_end_2]
    
    code_and_name_2 = stock_df_2['full_name'].str.replace('\u3000', ' ', regex=True).str.split(' ', expand=True)
    stock_df_2['code'] = code_and_name_2[0]

    print('Downloading data for strMode=4')
    # Request data for strMode=4
    res_4 = requests.get("https://isin.twse.com.tw/isin/C_public.jsp?strMode=4")
    df_4 = pd.read_html(res_4.text)[0]
    df_4.columns = [
        'full_name', 'isin_code', 'listed_date', 'market_type', 'industry_type', 'cfic_code', 'remarks'
    ]
    stock_start_4 = df_4[df_4['full_name'].str.contains('股票', na=False)].index[0] + 1
    stock_end_4 = df_4[df_4['full_name'].str.contains('特別股', na=False)].index[0]
    stock_df_4 = df_4.iloc[stock_start_4:stock_end_4]
    
    code_and_name_4 = stock_df_4['full_name'].str.replace('\u3000', ' ', regex=True).str.split(' ', expand=True)
    stock_df_4['code'] = code_and_name_4[0]

    # Combine results from both requests
    combined_df = pd.concat([stock_df_2[['code']], stock_df_4[['code']]], ignore_index=True)

    # Return the list of stock codes
    return combined_df['code'].tolist()

columns = ["broker", "branch", "code", "date", "buy", "sell", "total", "net"]

codes = get_stock_codes()
for code in codes:
    print(code)
    data = []
    for entry in broker_branches:
        broker_name = entry["broker_name"]
        branch_name = entry["branch_name"]
        broker_value = entry["broker_value"]
        branch_value = entry["branch_value"]

        # Construct the request URL
        data_url = f"https://concords.moneydj.com/z/zc/zco/zco0/zco0.djhtm?A={code}&BHID={broker_value}&b={branch_value}&C=1&D=2024-1-1&E=2025-1-22&ver=V3"

        while True:
            try:
                # Fetch the page content
                response = requests.get(data_url, headers=headers)
                if response.status_code != 200:
                    print(f"Failed to fetch data for {broker_name} - {branch_name}")
                    break

                # Parse the HTML content
                soup = BeautifulSoup(response.content, "html.parser")
                table = soup.find("table", {"id": "oMainTable"})
                if not table:
                    print(f"No table found for {broker_name} - {branch_name}")
                    break

                # Extract table rows
                rows = table.find_all("tr")[1:]  # Skip header row
                for row in rows:
                    cells = row.find_all("td")
                    if len(cells) >= 5:
                        data.append({
                            "broker": broker_name,
                            "branch": branch_name,
                            "code": code,
                            "date": cells[0].text.strip(),
                            "buy": int(cells[1].text.strip().replace(",", "")),
                            "sell": int(cells[2].text.strip().replace(",", "")),
                            "total": int(cells[3].text.strip().replace(",", "")),
                            "net": int(cells[4].text.strip().replace(",", ""))
                        })
                break

            except Exception as e:
                print(f"Error fetching data for {broker_name} - {branch_name}, retrying in 5 minutes: {e}")
                time.sleep(300)

    # Step 3: Store the data in a DataFrame
    df = pd.DataFrame(data, columns=columns)

    # Save the DataFrame to a CSV file
    df.to_csv(f"../data_sample/chip/broker_branch_data_{code}.csv", index=False, encoding="utf-8")